# Коллаборативная фильтрация

## Основная идея
Как понять, что пользователю $u$ нужно показать айтем $i$?
1. Если $u$ понравился айтем, похожий на $i$, то можно предположить, что ему понравится $i$.
2. Если ему не понравился айтем, похожий на $i$, ему, вероятно, не понравится и $i$.

Таким образом, идея состоит в том, чтобы посмотреть, насколько понравились $u$ айтемы, похожие на $i$.

## Item-to-item collaborative filtering

Реализуем на семинаре следующую вариацию item-to-item метода.
1. Пусть $U_i$ &mdash; множество пользователей, оценивших $i$. Определим, как мы будем определять похожести между айтемами. Определим похожести между айтемами, как
$$
         w(i, j) = \frac{\sum_{u\in U_i \cap U_j}(r_{u,i}-\bar{r}_u)(r_{u,j}-\bar{r}_u)}{\sqrt{\sum_{u\in U_i \cap U_j} (r_{u,i}-\bar{r}_u)^2}\sqrt{\sum_{u\in U_i \cap U_j} (r_{u,j}-\bar{r}_u)^2}}.
$$
Обозначим за $S_k^{(i)}$ множество из $k$ наиболее близких к $i$ айтемов.
   
2. Пользователю $u$, оценившему множество айтемов $I_u$, будем рекомендовать $N$ наиболее подходящих ему айтемов. Для этого,
- В качестве кандидатов на попадание в топ возьмем айтемы $$C = \bigcup_{i \in I_u} S_k^{(i)} \setminus I_u.$$
- Для кандидатов $c \in C$ определим похожесть $c$ на историю пользователя $I_u$, как
\begin{equation}
 w(c,I_u) = \sum_{i\in I_u} w_{c,i}.
\end{equation}
- Вернем в качестве результата $N$ айтемов $c \in C$ с максимальной величиной $w(c, I_u)$.


### Import useful requirements

In [15]:
import os
if not (os.path.exists("recsys.zip") or os.path.exists("recsys")):
    !wget https://github.com/nzhinusoftcm/review-on-collaborative-filtering/raw/master/recsys.zip    
    !unzip recsys.zip

In [16]:
from recsys.datasets import ml1m, ml100k
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm

import pandas as pd
import numpy as np
import os
import joblib
import sys

import typing as tp

In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Загрузка датасета

Для применения методов будем использовать датасет Movielens. Он представляет из себя оценки, которые пользователи поставили просмотренным фильмам и небольшие описания самих фильмов.

Для удобства изучения разных алгоритмов исследовательская группа, которая занимается разработкой датасета, подготовила данные разного объёма: 100к рейтингов, 1М, 10М, 20М. В этой работе мы будем пользоваться самым маленьким.

In [18]:
ratings, movies = ml100k.load()

In [19]:
ratings.head()

,userid,itemid,rating
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3


In [20]:
movies.head()

,itemid,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


### Предобработка датасета

In [21]:
def ids_encoder(ratings):
    users = sorted(ratings['userid'].unique())
    items = sorted(ratings['itemid'].unique())

    # create users and items encoders
    uencoder = LabelEncoder()
    iencoder = LabelEncoder()

    # fit users and items ids to the corresponding encoder
    uencoder.fit(users)
    iencoder.fit(items)

    # encode userids and itemids
    ratings.userid = uencoder.transform(ratings.userid.tolist())
    ratings.itemid = iencoder.transform(ratings.itemid.tolist())

    return ratings, uencoder, iencoder

In [22]:
# create the encoder
ratings, uencoder, iencoder = ids_encoder(ratings)

## Реализация алгоритма

### Шаг 1. Вычислить похожести между айтемами

#### adjustied cosine similarity

Вспомним, что каждый айтем можно представить как вектор оценок пользователей, тогда для оценки похожести можно использовать:
1. Косинусное расстояние
2. Ajusted Cosine Similarity
3. Евклидово расстояние
4. Манхэттенское расстояние
5. Коэффициент Жаккара



В item-based рекомендациях к-т adjustied cosine similarity доказал свою эффективность, поэтому будем использовать его. Схожесть между айтемами $i$ и $j$ считается по формуле:


\begin{equation}
 w_{i,j}= \frac{\sum_{u\in U}(r_{u,i}-\bar{r}_u)(r_{u,j}-\bar{r}_u)}{\sqrt{\sum_{u\in U} (r_{u,i}-\bar{r}_u)^2}\sqrt{\sum_{u\in U} (r_{u,j}-\bar{r}_u)^2}}.
\end{equation}

Итак, чтобы вычислить похожесть между айтемами $i$ и $j$, нужно

1. Выявить всех пользователей, которые оценили оба айтема, т.е. $U_i \cap U_j$;
2. Нормализовать рейтинги айтемов $i$ и $j$;
3. Посчитать похожесть между векторами общих рейтингов $i$ и $j$.


Проведём подготовительную работу и нормализуем рейтинги всех пользователей:

In [23]:
def normalize(ratings: pd.DataFrame) -> pd.DataFrame:
    """
    Нормализует рейтинги по пользователям. Из каждого рейтинга вычитает средний рейтинг по пользователю.
    ratings: таблица рейтингов
    
    Возвращает: 
        Таблица, содержащая все колонки таблицы `ratings` и колонку `norm_rating` с нормализованными рейтингами.
    """
    # your code here
    user_means = ratings[['userid', 'rating']].groupby('userid').mean().rename(columns={'rating': 'mean_rating'})
    ratings = ratings.join(user_means, on='userid')
    ratings['norm_rating'] = ratings.rating - ratings.mean_rating
    ratings.drop(columns=['mean_rating'], inplace=True)
    return ratings

In [24]:
def test_normalize():
    test_df = pd.DataFrame({
        "userid": [0, 0, 0, 1, 1],
        "itemid": [0, 1, 2, 1, 3],
        "rating": [2, 2, 5, 5, 5],
    })
    
    expected = pd.DataFrame({
        "userid": [0, 0, 0, 1, 1],
        "itemid": [0, 1, 2, 1, 3],
        "rating": [2, 2, 5, 5, 5],
        "norm_rating": [-1, -1, 2, 0, 0]
    })    
    
    assert test_df.shape[0] == expected.shape[0], "Number of user-item interactions is different"
    assert test_df.shape[1] + 1 == expected.shape[1], "Number of columns is incorrect"
    assert (normalize(test_df) == expected).all().all(), "Result is incorrect"
    
test_normalize()

In [25]:
norm_ratings = normalize(ratings)
np_ratings = norm_ratings.to_numpy()
norm_ratings.head()

,userid,itemid,rating,norm_rating
0,0,0,5,1.389706
1,0,1,3,-0.610294
2,0,2,4,0.389706
3,0,3,3,-0.610294
4,0,4,3,-0.610294


In [26]:
np_ratings.shape

(100000, 4)

Итак, мы вычислили средний рейтинг для каждого пользователя. Теперь мы готовы к вычислению похожестей айтемов.

In [27]:
def cosine(x: np.array, y: np.array) -> float:
    """
    Функция, вычисляющая косинус между векторами x и y.
    """
    if np.linalg.norm(x) == 0 or np.linalg.norm(y) == 0:
        return 0
    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

In [28]:
from functools import lru_cache

@lru_cache(2000)
def ratings_for_item(i):
    return np_ratings[np_ratings[:, 1] == i]

def calculate_similarity_between_two(np_ratings: np.array, i: int, j: int) -> float:
    """
    np_ratings: массив, каждый элемент которого является набором (user_id, item_id, rating, norm_rating)
    i: номер первого айтема для вычисления похожести
    j: номер второго айтема для вычисления похожести
    
    Возвращает значение adjustied cosine similarity для айтемов i и j.
    """
    # your code here
    
    if i == j:
        return 1
    
    ratings_i = ratings_for_item(i)
    ratings_j = ratings_for_item(j)
    
    common_users = np.intersect1d(ratings_i[:, 0], ratings_j[:, 0])
    common_ratings_i = ratings_i[np.isin(ratings_i[:, 0], common_users)]
    common_ratings_j = ratings_j[np.isin(ratings_j[:, 0], common_users)]
    
    if len(common_users) > 0:
        assert sorted(common_ratings_i[:, 0]) == sorted(common_ratings_j[:, 0])
        return cosine(common_ratings_i[:, 3], common_ratings_j[:, 3])
    
    return 0

In [29]:
ratings_i = ratings_for_item(1)
ratings_j = ratings_for_item(2)

common_users = np.intersect1d(ratings_i[:, 0], ratings_j[:, 0])
common_users

array([  0.,  48.,  94., 129., 206., 266., 267., 275., 279., 292., 300.,
       302., 319., 345., 392., 428., 434., 449., 471., 486., 496., 550.,
       560., 620., 621., 659., 681., 750., 794., 805., 879., 885., 888.,
       915.])

In [14]:
assert np.isclose(calculate_similarity_between_two(np_ratings, 0, 0), 1.0)
assert np.isclose(calculate_similarity_between_two(np_ratings, 1, 2), 0.1069226)
assert np.isclose(calculate_similarity_between_two(np_ratings, 1, 3), 0.0555092)
assert np.isclose(calculate_similarity_between_two(np_ratings, 1, 5), -0.125509)
assert np.isclose(calculate_similarity_between_two(np_ratings, 1, 1431), 1.0)
assert np.isclose(calculate_similarity_between_two(np_ratings, 4, 1123), 0.0)

In [32]:
def adjusted_cosine(np_ratings: np.array, similarity_between_two) -> tp.Tuple[np.array, np.array]:
    """
    Функция, вычисляющая adjustied cosine similarity для всевозможных пар айтемов (i, j).
    
    np_ratings: массив, каждый элемент которого является набором (user_id, item_id, rating, norm_rating)
    similarity_between_two: функция для подсчёта похожестей между двумя айтемами, принимает на вход массив с рейтингами и айди айтемов для подсчёта похожестей
    
    Возвращает:
        1. массив размера |I|x|I|, в i-ой строке которого расположены в порядке убывания похожести i-го айтема
        2. массив размера |I|x|I|, в i-ой строке которого расположены айди айтемов 
            в порядке убывания их похожестей с i-ым айтемом
    """
    nb_items = np.unique(np_ratings[:, 1]).size
    similarities = np.zeros(shape=(nb_items, nb_items))
    similarities.fill(-1) # ...
    
    np.fill_diagonal(similarities, 1)
    items = sorted(set(map(int, np_ratings[:, 1])))
    
    for i in tqdm(range(len(items))):
        for j in range(i + 1, len(items)):
            # your code here
            
            similarity = np.clip(similarity_between_two(np_ratings, items[i], items[j]), -1, 1) # clip для ассертов, а вообще не обязателен
            similarities[items[i], items[j]] = similarity
            similarities[items[j], items[i]] = similarity
            
    assert np.all(similarities.T == similarities), 'Similarity matrix should be symmetrical'
    assert np.allclose(np.diag(similarities), 1.0), 'Similarities of items with themselves should be 1'
    
    # get neighbors by their neighbors in decreasing order of similarities
    neighbors = np.flip(np.argsort(similarities), axis=1)
    
    # sort similarities in decreasing order
    similarities = np.flip(np.sort(similarities), axis=1)
        
    return similarities, neighbors

Вспользуемся написанной функцией:

In [33]:
similarities, neighbors = adjusted_cosine(np_ratings, calculate_similarity_between_two)

  0%|          | 0/1682 [00:00<?, ?it/s]

In [34]:
def sorted_neighbors(similarities, neighbors):
    return [i[1] for i in sorted(zip(similarities, neighbors), key=lambda x: (x[0], -x[1]))][::-1]

assert np.equal(sorted_neighbors(similarities[1], neighbors[1])[:10], 
            [1, 295, 307, 313, 358, 642, 705, 756, 821, 829]).all()
assert np.equal(sorted_neighbors(similarities[2], neighbors[2])[:10], 
            [2, 112, 118, 313, 343, 376, 453, 535, 591, 593]).all() 
assert np.equal(sorted_neighbors(similarities[201], neighbors[201])[:10], 
            [201, 360, 598, 676, 813, 847, 1079, 1095, 1105, 1122]).all()
assert np.equal(sorted_neighbors(similarities[800], neighbors[800])[:10], 
            [9, 18, 33, 35, 36, 44, 56, 73, 103, 212]).all()

Посмотрим глазами на списки соседей, которые мы получили.

In [35]:
def neighbours_viz(item_id: int, movies: pd.DataFrame,
                          similarities: np.array, neighbours: np.array, k=5):
    """
    item_id: id фильма, для которого вычисляются соседи
    movies: таблица с данными о фильмах
    similarities: массив похожестей
    neighbours: массив соседей для всех айтемов
    """
    film_name = movies[movies.itemid == iencoder.inverse_transform([item_id])[0]].title.values[0]
    similar_films = (
        (neighbor_id, movies[movies.itemid == iencoder.inverse_transform([neighbor_id])[0]].title.values[0], similarity)
        for neighbor_id, similarity in zip(neighbors[item_id][:k], similarities[item_id][:k])
    )
    display(pd.DataFrame(dict(zip(('item_id', film_name, 'Similarity'), zip(*similar_films)))))
    print('\n')

In [36]:
neighbours_viz(49, movies, similarities, neighbors)
neighbours_viz(68, movies, similarities, neighbors)
neighbours_viz(154, movies, similarities, neighbors)
neighbours_viz(200, movies, similarities, neighbors)

,item_id,Star Wars (1977),Similarity
0,1634,Two Friends (1986),1.0
1,1195,"Savage Nights (Nuits fauves, Les) (1992)",1.0
2,1462,"Boys, Les (1997)",1.0
3,1646,Hana-bi (1997),1.0
4,1645,Men With Guns (1997),1.0


,item_id,Forrest Gump (1994),Similarity
0,1293,Ayn Rand: A Sense of Life (1997),1.0
1,1640,Dadetown (1995),1.0
2,1157,"Fille seule, La (A Single Girl) (1995)",1.0
3,1331,My Life and Times With Antonin Artaud (En comp...,1.0
4,1462,"Boys, Les (1997)",1.0


,item_id,Dirty Dancing (1987),Similarity
0,1681,Scream of Stone (Schrei aus Stein) (1991),1.0
1,1143,"Quiet Room, The (1996)",1.0
2,1552,"Underneath, The (1995)",1.0
3,1551,"Hunted, The (1995)",1.0
4,1081,Female Perversions (1996),1.0


,item_id,Evil Dead II (1987),Similarity
0,888,"Tango Lesson, The (1997)",1.0
1,1552,"Underneath, The (1995)",1.0
2,1561,"Eye of Vichy, The (Oeil de Vichy, L') (1993)",1.0
3,1560,Tigrero: A Film That Was Never Made (1994),1.0
4,1559,Clean Slate (Coup de Torchon) (1981),1.0


In [37]:
len(np_ratings[np_ratings[:, 1] == 1634])

1

Как вам кажется, получились ли у нас хорошие результаты? Что объединяет нерелевантные айтемы из топов по похожестям?

In [38]:
def calculate_similarity_between_two_with_threshold(np_ratings: np.array, i: int, j: int) -> float:
    """
    np_ratings: массив, каждый элемент которого является набором (user_id, item_id, rating, norm_rating)
    i: номер первого айтема для вычисления похожести
    j: номер второго айтема для вычисления похожести

    Возвращает значение adjustied cosine similarity для айтемов i и j. 
    """
    THRESHOLD = 20 # порог для числа общих юзеров. считаем пару валидной если общее число больше порога
    # your code here
    
    i_ratings = ratings_for_item(i)
    j_ratings = ratings_for_item(j)
    common_users = np.intersect1d(i_ratings[:, 0].astype(int), j_ratings[:, 0].astype(int))
    
    if len(common_users) <= THRESHOLD:
        return 0
    
    i_ratings = i_ratings[np.isin(i_ratings[:, 0].astype(int), list(common_users))]
    j_ratings = j_ratings[np.isin(j_ratings[:, 0].astype(int), list(common_users))]
    similarity = cosine(i_ratings[:, 3], j_ratings[:, 3])
    
    return similarity

In [39]:
assert np.isclose(calculate_similarity_between_two_with_threshold(np_ratings, 1, 1431), 0)
assert np.isclose(calculate_similarity_between_two_with_threshold(np_ratings, 1, 17), 0)
assert np.isclose(calculate_similarity_between_two_with_threshold(np_ratings, 4, 1123), 0)
assert np.isclose(calculate_similarity_between_two_with_threshold(np_ratings, 914, 1681), 0)

In [40]:
calculate_similarity_between_two_with_threshold(np_ratings, 49, 840)

-0.6264337107802631

In [41]:
similarities, neighbors = adjusted_cosine(np_ratings, calculate_similarity_between_two_with_threshold)

  0%|          | 0/1682 [00:00<?, ?it/s]

Посмотрим, что получилось теперь:

In [42]:
neighbours_viz(49, movies, similarities, neighbors)
neighbours_viz(68, movies, similarities, neighbors)
neighbours_viz(154, movies, similarities, neighbors)
neighbours_viz(200, movies, similarities, neighbors)

,item_id,Star Wars (1977),Similarity
0,49,Star Wars (1977),1.000000
1,171,"Empire Strikes Back, The (1980)",0.826287
2,180,Return of the Jedi (1983),0.728182
3,173,Raiders of the Lost Ark (1981),0.714250
4,407,"Close Shave, A (1995)",0.659379


,item_id,Forrest Gump (1994),Similarity
0,68,Forrest Gump (1994),1.000000
1,214,Field of Dreams (1989),0.445981
2,309,"Rainmaker, The (1997)",0.436435
3,21,Braveheart (1995),0.422572
4,965,"Affair to Remember, An (1957)",0.420235


,item_id,Dirty Dancing (1987),Similarity
0,154,Dirty Dancing (1987),1.000000
1,626,Robin Hood: Prince of Thieves (1991),0.727235
2,568,Wolf (1994),0.716361
3,254,My Best Friend's Wedding (1997),0.708146
4,416,"Parent Trap, The (1961)",0.634029


,item_id,Evil Dead II (1987),Similarity
0,200,Evil Dead II (1987),1.000000
1,183,Army of Darkness (1993),0.574317
2,23,Rumble in the Bronx (1995),0.536677
3,90,"Nightmare Before Christmas, The (1993)",0.491057
4,557,Heavenly Creatures (1994),0.481768


### Шаг 2. Выбрать топ рекомендаций для пользователя

Теперь, когда для каждого айтема есть список похожих, научимся формировать рекомендацию для пользователя.

#### Отбор кандидатов

1. Для каждого айтема из истории соберём сет из k самых похожих айтемов
2. Объединим полученные сеты
3. Уберём из полученного множества те айтемы, с которыми пользователь уже взаимодействовал

In [43]:
def candidate_items(np_ratings: np.array, userid: int, k=-1) -> tp.Tuple[np.array, np.array]:
    """
    np_ratings: массив, каждый элемент которого является набором (user_id, item_id, rating, norm_rating)
    userid: id пользователя, для которого генерируются кандидаты
    k: количество кандидатов с каждого айтема из истории
    
    Возвращает 
        1. массив user_item_ids с id фильмов, просмотренных пользователем
        2. массив айтемов, близких к айтемам из истории пользователя
    """
    
    # your code here
    
    user_item_ids = np_ratings[np_ratings[:, 0] == userid]
    user_item_ids = user_item_ids[:, 1].astype(int)
    
    c = set()
    
    for item_id in user_item_ids:
        c.update(neighbors[item_id, :k])
        
    c = list(c)
    
    candidates = np.setdiff1d(c, user_item_ids, assume_unique=True)
    
    return user_item_ids, candidates

In [44]:
user_item_ids, u_candidates = candidate_items(np_ratings, uencoder.transform([3])[0])

print('Количество просмотренных фильмов пользователя 1:', len(user_item_ids))
print('Количество кандидатов для пользователя 1:', len(u_candidates))

Количество просмотренных фильмов пользователя 1: 54
Количество кандидатов для пользователя 1: 1628


In [46]:
user_item_ids_test, u_candidates_test = candidate_items(np_ratings, uencoder.transform([1])[0])
assert len(user_item_ids_test) == 272
assert len(u_candidates_test) == 1410

user_item_ids_test, u_candidates_test = candidate_items(np_ratings, uencoder.transform([50])[0])
assert len(user_item_ids_test) == 24
assert len(u_candidates_test) == 1658

user_item_ids_test, u_candidates_test = candidate_items(np_ratings, uencoder.transform([200])[0], 30)
assert len(user_item_ids_test) == 216
assert len(u_candidates_test) == 607

user_item_ids_test, u_candidates_test = candidate_items(np_ratings, uencoder.transform([200])[0], 15)
assert len(user_item_ids_test) == 216
assert len(u_candidates_test) == 526

user_item_ids_test, u_candidates_test = candidate_items(np_ratings, uencoder.transform([942])[0])
assert len(user_item_ids_test) == 79
assert len(u_candidates_test) == 1603

del user_item_ids_test, u_candidates_test

#### Вычисление похожести между кандидатом и множеством айтемов из истории пользователя u

In [47]:
def similarity_with_user_items(item_id: int, user_item_ids: np.array, similarities: np.array, neighbors: np.array) -> float:
    """
    item_id: id айтема-кандидата, для которого считается похожесть с историей пользователя
    user_item_ids: массив id фильмов, просмотренных пользователем
    similarities: массив похожестей айтемов
    neighbors: массив соседей для всех айтемов
    
    Возвращает число – похожесть айтема на историю пользователя.
    """
    w = 0     
    for i_id in user_item_ids:        
        # get similarity between itemid and c, if c is one of the k nearest neighbors of itemid
        # your code here
        
        if i_id in neighbors[item_id]:
            w = w + similarities[i_id, neighbors[i_id] == item_id][0]
    return w

In [48]:
user_item_ids_test, _ = candidate_items(np_ratings, uencoder.transform([1])[0])

assert np.isclose(similarity_with_user_items(0, user_item_ids_test, similarities, neighbors), 9.852485)
assert np.isclose(similarity_with_user_items(200, user_item_ids_test, similarities, neighbors), 5.914738)

user_item_ids_test, _ = candidate_items(np_ratings, uencoder.transform([300])[0])

assert np.isclose(similarity_with_user_items(200, user_item_ids_test, similarities, neighbors), 0.294773)
assert np.isclose(similarity_with_user_items(242, user_item_ids_test, similarities, neighbors), 5.008178)

#### Ранжирование кандидатов по их похожестям на историю пользователя

In [49]:
def rank_candidates(candidates: np.array, user_item_ids: np.array, similarities: np.array, neighbors: np.array) -> np.array:
    """
    candidates: массив id фильмов-кандидатов
    user_item_ids: массив id фильмов, просмотренных пользователем
    similarities: массив похожестей айтемов
    neighbors: массив соседей для всех айтемов
    
    Возвращает массив tuple, где первый элемент – айди айтема, второй – похожесть на историю пользователя
    """
    
    # list of candidate items mapped to their corresponding similarities to user_item_ids
    sims = [similarity_with_user_items(c, user_item_ids, similarities, neighbors) for c in candidates]
    candidates = iencoder.inverse_transform(candidates)    
    mapping = list(zip(candidates, sims))
    
    ranked_candidates = sorted(mapping, key=lambda couple:couple[1], reverse=True)    
    return ranked_candidates

In [50]:
user_item_ids_test, candidates_test = candidate_items(np_ratings, uencoder.transform([1])[0])
assert len(rank_candidates(candidates_test, user_item_ids_test, similarities, neighbors)) == len(candidates_test)
assert rank_candidates(candidates_test, user_item_ids_test, similarities, neighbors)[0][0] == 408
assert rank_candidates(candidates_test, user_item_ids_test, similarities, neighbors)[10][0] == 792
assert rank_candidates(candidates_test, user_item_ids_test, similarities, neighbors)[19][0] == 661

## Соберём всё вместе

Теперь у нас есть всё, что нужно: отбор кандидатов, ранжирующая функция и мы готовы собрать весь пайплайн item-to-item рекомендаций.

In [55]:
def topn_recommendation(np_ratings: np.array, userid: int, similarities: np.array, neighbors: np.array, k=-1, N=30):
    """
    np_ratings: массив, каждый элемент которого является набором (user_id, item_id, rating, norm_rating)
    userid: id пользователя, для которого генерируются кандидаты
    similarities: массив похожестей айтемов
    neighbors: массив соседей для всех айтемов
    k: количество кандидатов на стадии отбора кандидатов
    N: количество рекомендаций фильмов для пользователя
    
    Возвращает dataframe c рекомендацией top-N фильмов для пользователя userid.
    """
    # find candidate items
    # user_item_ids, candidates = # your code here
    user_item_ids, candidates = candidate_items(np_ratings, userid, k)
    
    
    # rank candidate items according to their similarities with user_item_ids
    # ranked_candidates = # your code here
    ranked_candidates = rank_candidates(candidates, user_item_ids, similarities, neighbors)
    
    
    # get the first N row of ranked_candidates to build the top N recommendation list
    topn = pd.DataFrame(ranked_candidates[:N], columns=['itemid', 'similarity_with_Iu'])    
    topn = pd.merge(topn, movies, on='itemid', how='inner')    
    return topn

Посмотрим, как это работает:

In [56]:
topn_recommendation(np_ratings, uencoder.transform([1])[0], similarities, neighbors)

,itemid,similarity_with_Iu,title
0,408,26.691274,"Close Shave, A (1995)"
1,302,26.377228,L.A. Confidential (1997)
2,316,23.746689,As Good As It Gets (1997)
3,467,22.908033,"Bronx Tale, A (1993)"
4,963,21.545264,Some Folks Call It a Sling Blade (1993)
5,293,20.832552,Donnie Brasco (1997)
6,313,20.058310,Titanic (1997)
7,1039,19.332570,Hamlet (1996)
8,750,19.318480,Amistad (1997)
9,315,18.907395,Apt Pupil (1998)


А теперь попробуем применить то, что у нас получилось на тестовом пользователе, для которого соберём историю просмотров сами.

In [57]:
test_history = [49, 81, 180, 256, 131, 379]
movies.iloc[test_history]

,itemid,title
49,50,Star Wars (1977)
81,82,Jurassic Park (1993)
180,181,Return of the Jedi (1983)
256,257,Men in Black (1997)
131,132,"Wizard of Oz, The (1939)"
379,380,Star Trek: Generations (1994)


In [58]:
def topn_recommendations_by_user_history(user_item_ids: tp.List[int], similarities: np.array, neighbors: np.array, k=-1, N=30):
    c = set()
    for i_id in user_item_ids:    
        c.update(neighbors[i_id, :k])
    candidates = np.setdiff1d(list(c), user_item_ids, assume_unique=True)
    
    ranked_candidates = rank_candidates(candidates, user_item_ids, similarities, neighbors)
    topn = pd.DataFrame(ranked_candidates[:N], columns=['itemid','similarity_with_Iu'])    
    topn = pd.merge(topn, movies, on='itemid', how='inner')
    return topn

topn_recommendations_by_user_history(test_history, similarities, neighbors)

,itemid,similarity_with_Iu,title
0,172,2.628615,"Empire Strikes Back, The (1980)"
1,174,2.142320,Raiders of the Lost Ark (1981)
2,313,2.035859,Titanic (1997)
3,210,1.941583,Indiana Jones and the Last Crusade (1989)
4,651,1.880540,Glory (1989)
5,966,1.829362,"Affair to Remember, An (1957)"
6,22,1.773043,Braveheart (1995)
7,64,1.735550,"Shawshank Redemption, The (1994)"
8,79,1.707949,"Fugitive, The (1993)"
9,963,1.703303,Some Folks Call It a Sling Blade (1993)


Итак, в этом семинаре мы научились строить item-to-item рекомендации. Этот подход можно улучшать и развивать. 

1. Например, мы можем учитывать рейтинги айтемов из истории пользователя. Определить, когда рейтинг был позитивный и учитывать кандидатов только для таких айтемов
2. Можем поэксеприментировать над определением похожести